In [ ]:
# import os
from pathlib import Path
import time
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from eemeter.development.model import HourlyOptData, HourlyOptModel
import pickle
import multiprocessing as mp

# from hourly_test_utils import *
from applied_data_science.bigquery.data import Meter_Data
from eemeter import eemeter as em
from eemeter.common.metrics import BaselineTestingMetrics as Metrics

import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2

In [ ]:
#load subsamples from the main MCE dataset
dataset = 'mce_3_yr_precovid'
subsample = 1
has_solar = True

cache_dir = Path("/app/.recurve_cache/data").resolve()

if 'data' in globals():
    del data

data = Meter_Data(dataset, subsample, "hourly", solar=has_solar, cache_dir=cache_dir)

meta = data.df['meta']
subsample_df = data.df['meter']
ids = subsample_df.index.unique()

In [ ]:
data_settings = em.HourlySettings(
    TRAIN_FEATURES=['ghi'],
    SUPPLEMENTAL_DATA= {'TS_SUPPLEMENTAL': None,
                        'CATEGORICAL_SUPPLEMENTAL': {'PV_INSTALLATION_DATE': "2019-05-14"},},
)

alpha_opt = 0.15
l1_ratio_opt = 0.05
model_settings = em.HourlySettings(
    ALPHA=alpha_opt,
    L1_RATIO=l1_ratio_opt,
    SELECTION="cyclic",
    MAX_ITER=1000,
    SEED=42
)
months = [
    "jan",
    "feb",
    "mar",
    "apr",
    "may",
    "jun",
    "jul",
    "aug",
    "sep",
    "oct",
    "nov",
    "dec",
]


### Getting features for each fold of individual id

In [ ]:
def _each_month_data_dec(arglst):
    non_neccessary_columns = ['jan_train', 'jan_test',
       'feb_train', 'feb_test', 'mar_train', 'mar_test', 'apr_train',
       'apr_test', 'may_train', 'may_test', 'jun_train', 'jun_test',
       'jul_train', 'jul_test', 'aug_train', 'aug_test', 'sep_train',
       'sep_test', 'oct_train', 'oct_test', 'nov_train', 'nov_test',
       'dec_train', 'dec_test']
    X_train, y_train, X_test, y_test = [], [], [], []
    df, month, kwargs = arglst
    idx_train = np.argwhere(np.isfinite(df[f"{month}_train"].values)).flatten()

    idx_finite = np.argwhere(
        np.isfinite(df.iloc[idx_train]["observed"].values)
    ).flatten()
    if len(idx_finite) == 0:
        pass

    idx_train = idx_train[idx_finite]

    idx_test = np.argwhere(np.isfinite(df[f"{month}_test"].values)).flatten()

    idx_finite = np.argwhere(
        np.isfinite(df.iloc[idx_test]["observed"].values)
    ).flatten()

    if len(idx_finite) == 0:
        pass

    idx_test = idx_test[idx_finite]

    df_train_temp = df.iloc[idx_train]
    df_test_temp = df.iloc[idx_test]

    df_train_temp['model'] = df_train_temp[f"{month}_{'train'}"]
    df_test_temp['model'] = df_test_temp[f"{month}_{'test'}"]

    df_train_temp = df_train_temp.drop(columns=non_neccessary_columns)
    df_test_temp = df_test_temp.drop(columns=non_neccessary_columns)

    train_data = HourlyOptData(df_train_temp, data_settings, **kwargs)
    y_scaler = train_data._y_scaler

    X_train, y_train = train_data.X, train_data.y

    test_data = HourlyOptData(df_test_temp, data_settings, **kwargs)
    X_test, y_test = test_data.X, test_data.y

    return X_train, y_train, df_train_temp['model'].values, X_test, y_test, df_test_temp['model'].values, y_scaler

def get_kfold_data_3_years(meta_id, df, kwargs):
    arglst = [(df, month, kwargs) for month in months]
    with mp.Pool(mp.cpu_count()) as pool:
        res = pool.map(_each_month_data_dec, arglst)
    return meta_id, res

## Pickle a subsample features

@Travis if you can use multiprocessing to speed up the process, that would be great. I tried mp for id level but for some reason it wasn't faster
for me using mp worked better when I used it for each month level (3 times faster)


Save the data with the following cell, and if you've already done it just skip it (I forgot to do that in my VM one time, which takes 57 minutes for one subsample)

In [ ]:
subsample_features = []
for meter_id in ids:
    meter = subsample_df.loc[meter_id].copy()
    meta_id = meta.loc[meter_id].copy()
    unique_pairs = meta_id[['station_latitude', 'station_longitude']].drop_duplicates()
    METADATA = {
                'station_latitude': unique_pairs['station_latitude'].values[0],
                'station_longitude': unique_pairs['station_longitude'].values[0],
                    }
    OUTPUT_FEATURES=['temperature', 'observed', 'model']
    if 'ghi' in data_settings.TRAIN_FEATURES:
            CONSIDER_SOLAR = True
            OUTPUT_FEATURES.append('ghi')
            OUTPUT_FEATURES.append('clearsky_ghi')
    else:
        CONSIDER_SOLAR = False
    kwargs = {
        'solar': CONSIDER_SOLAR,
        'metadata': METADATA,
        'outputs': OUTPUT_FEATURES,
    }
    res = get_kfold_data_3_years(meta_id, meter, kwargs)
    subsample_features.append(res)

subsample_results_saving_path = Path("/app/.recurve_cache/data/subsample_features").resolve()
subsample_results_saving_path.mkdir(exist_ok=True)
import pickle
with open(subsample_results_saving_path / f"subsample_features_{dataset}_{subsample}.pkl", "wb") as f:
    pickle.dump(subsample_features, f)

## Trining the models

Read the subsample features here
Also you can model them as well here!

Have that in mind there is a missing_values_amount variable in HourlyOptData which shows if we exceed the 10% threshold during interpolation.
Then if that's the case we won't have features at all and all of them would be None (like y_train = None)
And if this is None then we skip it in the modeling. So you can count the number of "OK" ids if you want.

Right now I'm taking the average of test PNRMSE for those who actually have values in the features, in the objective funciton.

In [ ]:
subsample_results_saving_path = Path("/app/.recurve_cache/data/subsample_features").resolve()
with open(subsample_results_saving_path / f"subsample_features_{dataset}_{subsample}.pkl", "rb") as f:
    subsample_features = pickle.load(f)

def id_model_dec(arglsts):
    meter_id, kfold_features = arglsts
    all_id_test_error = []
    for i in range(len(kfold_features)):
        X_train, y_train, oeem_train, X_test, y_test, oeem_test, y_scaler = kfold_features[i]
        if y_train is None: 
            # print('bad id: ', meter_id.index.unique()[0])
            break
        model = HourlyOptModel(model_settings)
        y_pred_train = model.fit_predict(X_train, y_train)
        y_predict_train = y_scaler.inverse_transform(y_pred_train)
        y_predict_train = y_predict_train.flatten()

        y_train_inv = y_scaler.inverse_transform(y_train)
        y_train_inv = y_train_inv.flatten()

        y_pred_test = model.predict(X_test)
        y_predict_test = y_scaler.inverse_transform(y_pred_test)
        y_predict_test = y_predict_test.flatten()

        y_test_inv = y_scaler.inverse_transform(y_test)
        y_test_inv = y_test_inv.flatten()

        iqr = np.percentile(y_train_inv, 75) - np.percentile(y_train_inv, 25)
        rmse = np.sqrt(np.mean((y_predict_test - y_test_inv) ** 2))
        test_pnrmse = rmse / iqr

        all_id_test_error.append(test_pnrmse)
    
    id_test_pnrmse = np.mean(all_id_test_error)
    # print(id_test_pnrmse)
    return id_test_pnrmse

# for i in range(len(subsample_features)):
#     lst = subsample_features[i]
#     print(i)
#     print(id_model_dec(lst))

with mp.Pool(mp.cpu_count()) as pool:
    model_res = pool.map(id_model_dec, subsample_features)


### Optimization here!

In [ ]:
def obj_fcn(X, gradient=np.array([])):
    timer_start = time.time()

    alpha, l1_ratio = X

    model_settings.ALPHA = alpha
    model_settings.L1_RATIO = l1_ratio
    
    with mp.Pool(mp.cpu_count()-1) as pool:
        model_res = pool.map(id_model_dec, subsample_features)

    model_res = np.array(model_res)
    #remove nan values
    model_res = model_res[~np.isnan(model_res)]
    
    obj = np.mean(model_res)
    print(f"alpha: {alpha}, l1_ratio: {l1_ratio}, obj: {obj}")
    print(f"Time taken: {time.time()-timer_start}")
    return float(obj)

In [ ]:
# Optimization settings
import rbfopt

opt_options = {
    "global": {"algorithm": "RBFOpt", 
               "stop_criteria_type": 'Iteration Maximum', 
               "stop_criteria_val": 2, 
               "initial_step": 0.01, # percentage},
               "xtol_rel": 1E-5,
               "ftol_rel": 1E-5,
               "initial_pop_multiplier": 2,
    },
}

x0 = [0.01, 0.01]
bnds = [(0.0001, 1), (0.0001, 1)]


bnds = np.array(bnds).T
n_dim = np.size(bnds[0])

var_type = ['R']*n_dim  
max_eval = 400
max_time = 1E30
rbfopt_initialize = True
bb = rbfopt.RbfoptUserBlackBox(n_dim, np.array(bnds[0]), np.array(bnds[1]),
                                np.array(var_type), obj_fcn)


bonmin_path = "/app/applied_data_science/tools/optimization/coin-or/bonmin"
ipopt_path = "/app/applied_data_science/tools/optimization/coin-or/ipopt"
rbfsettings = rbfopt.RbfoptSettings(max_iterations=max_eval,
                                    max_evaluations=max_eval,
                                    max_cycles=1E30,
                                    max_clock_time=max_time,
                                    minlp_solver_path=bonmin_path, 
                                    nlp_solver_path=ipopt_path,)
                                    
algo = rbfopt.RbfoptAlgorithm(rbfsettings, bb, init_node_pos=x0, do_init_strategy=rbfopt_initialize)
loss, x_opt, itercount, evalcount, fast_evalcount = algo.optimize()
